In [ ]:
def get_edge_colors(image, sample_width):
    """Extract colors from the edges of the image."""
    top_edge = image[:sample_width, :]
    bottom_edge = image[-sample_width:, :]
    left_edge = image[:, :sample_width]
    right_edge = image[:, -sample_width:]

    edges = np.concatenate([top_edge.reshape(-1, 3), bottom_edge.reshape(-1, 3), left_edge.reshape(-1, 3), right_edge.reshape(-1, 3)], axis=0)
    return edges

def apply_edge_colors_border(image, border_size, edges):
    """Apply edge colors to the border of the image."""
    border_image = np.zeros(
        (image.shape[0] + 2 * border_size, image.shape[1] + 2 * border_size, 3), dtype=np.uint8)

    # Randomly select edge colors for the entire border area
    random_edge_colors = edges[np.random.randint(0, edges.shape[0], size=(border_image.shape[0], border_image.shape[1]))]

    # Assign random colors to all border areas
    border_image[:border_size, :] = random_edge_colors[:border_size, :]
    border_image[-border_size:, :] = random_edge_colors[-border_size:, :]
    border_image[:, :border_size] = random_edge_colors[:, :border_size]
    border_image[:, -border_size:] = random_edge_colors[:, -border_size:]

    # Insert the original image in the center of the border image
    border_image[border_size:-border_size, border_size:-border_size] = image
    
    return border_image

def pad_with_custom_border(image, min_height, min_width):
    """Pad the image to the target size with border based on edge colors."""
    # Apply padding with a constant color (initially)
    pad_transform = A.PadIfNeeded(min_height=min_height, min_width=min_width, border_mode=cv2.BORDER_CONSTANT, value=[0, 0, 0])
    padded_image = pad_transform(image=image)['image']
    
    # Extract edge colors from the original image (or the padded image)
    sample_width = 10
    edges = get_edge_colors(image, sample_width)
    
    # Apply the custom border using edge colors
    return apply_edge_colors_border(padded_image, border_size=min_height - image.shape[0], edges=edges)

# Define the transformation pipeline
transform = A.Compose([A.Lambda(image=lambda img, **kwargs: pad_with_custom_border(img, min_height=1107, min_width=1107))])

# Example usage
image = cv2.imread('your_image.jpg')
augmented = transform(image=image)
cv2.imwrite('augmented_image.jpg', augmented['image'])


In [ ]:
    elif aug == 'EPT-biomass-test':
        def get_edge_colors(image, sample_width):
            """Extract colors from the edges of the image."""
            top_edge = image[:sample_width, :]
            bottom_edge = image[-sample_width:, :]
            left_edge = image[:, :sample_width]
            right_edge = image[:, -sample_width:]

            edges = np.concatenate([top_edge.reshape(-1, 3), 
                                    bottom_edge.reshape(-1, 3), 
                                    left_edge.reshape(-1, 3), 
                                    right_edge.reshape(-1, 3)], 
                                   axis=0)
            return edges

        def apply_edge_colors_border(image, border_size, edges):
            """Apply edge colors to the border of the image."""
            border_image = np.zeros(
                (image.shape[0] + 2 * border_size, image.shape[1] + 2 * border_size, 3), dtype=np.uint8)

            # Randomly select edge colors for the entire border area
            random_edge_colors = edges[np.random.randint(0, edges.shape[0], size=(border_image.shape[0], border_image.shape[1]))]

            # Assign random colors to all border areas
            border_image[:border_size, :] = random_edge_colors[:border_size, :]
            border_image[-border_size:, :] = random_edge_colors[-border_size:, :]
            border_image[:, :border_size] = random_edge_colors[:, :border_size]
            border_image[:, -border_size:] = random_edge_colors[:, -border_size:]

            # Insert the original image in the center of the border image
            border_image[border_size:-border_size, border_size:-border_size] = image
            
            return border_image

        def pad_with_custom_border(image, min_height, min_width):
            """Pad the image to the target size with border based on edge colors."""
            # Apply padding with a constant color (initially)
            pad_transform = A.PadIfNeeded(min_height=min_height, min_width=min_width, 
                                          border_mode=cv2.BORDER_CONSTANT, value=[0, 0, 0])
            padded_image = pad_transform(image=image)['image']

            # Extract edge colors from the original image (or the padded image)
            sample_width = 10
            edges = get_edge_colors(image, sample_width)

            # Apply the custom border using edge colors
            return apply_edge_colors_border(padded_image, border_size=min_height - image.shape[0], edges=edges)
        
        transform_test = A.Compose([
                A.Lambda(image=lambda img, **kwargs: pad_with_custom_border(img, min_height=1107, min_width=1107)),
                A.Resize(imsize, imsize, p=1.0),
                a_end_tf,
        ])
        transform_train = A.Compose([
                A.Lambda(image=lambda img, **kwargs: pad_with_custom_border(img, min_height=1107, min_width=1107)),
                A.Flip(),
                A.RandomRotate90(p=0.5),
                A.Resize(imsize, imsize, p=1.0),
                a_end_tf,
        ])
        tf_test = lambda x: transform_test(image=np.array(x))["image"]
        tf_train = lambda x: transform_train(image=np.array(x))["image"]